### ### PydanticOutputParser을 이용한 기사 요약


In [ ]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
# OPENAI_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv() 

True

In [13]:
from typing import List, Optional

# 1. 기사에서 추출할 정보를 담을 Pydantic 모델 정의
# Pydantic BaseModel을 상속받아 데이터의 구조와 타입을 명확하게 정의합니다.
class TeamStanding(BaseModel):
    """
    팀의 현재 순위 정보를 나타냅니다.
    """
    team: str = Field(description="팀 이름")
    wins: int = Field(description="승리 횟수")
    losses: int = Field(description="패배 횟수")
    rank: int = Field(description="순위")
    score_difference: Optional[str] = Field(description="세트 득실 정보", default=None)

class MatchInfo(BaseModel):
    """
    다가오는 경기 정보를 나타냅니다.
    """
    team: str = Field(description="경기를 치르는 팀 이름")
    opponent: str = Field(description="상대 팀 이름")
    date: str = Field(description="경기 날짜")

class LCKArticleSummary(BaseModel):
    """
    주어진 기사의 핵심 내용을 요약합니다.
    """
    article_title: str = Field(description="기사 제목")
    main_topic: str = Field(description="기사의 핵심 주제")
    teams_involved: List[str] = Field(description="기사에서 언급된 팀들의 이름")
    current_standings: List[TeamStanding] = Field(description="기사에 언급된 팀들의 현재 순위")
    upcoming_matches: List[MatchInfo] = Field(description="기사에 언급된 주요 다가오는 경기들")

# 2. PydanticOutputParser 인스턴스 생성
# 이 파서가 LCKArticleSummary 모델의 구조에 맞춰 LLM의 출력을 파싱하도록 설정합니다.
parser = PydanticOutputParser(pydantic_object=LCKArticleSummary)

# 3. LangChain 구성 요소 정의
# LLM 모델 로드 (OpenAI 예제)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=os.environ.get("OPENAI_API_KEY"))

# 프롬프트 템플릿 정의
# 사용자에게 기사를 제공하고, Pydantic 모델에 맞는 JSON 출력을 요청하는 지시사항을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Provide a JSON object that summarizes the following article based on the provided Pydantic schema.\n"
            "{format_instructions}"
        ),
        (
            "human",
            "Article:\n{article_text}"
        ),
    ]
)

# 4. LangChain Expression Language (LCEL) 체인 구축
# LLM, 프롬프트, 파서를 순서대로 연결하여 전체 워크플로우를 정의합니다.
chain = prompt | llm | parser

# 5. 분석할 기사 내용
article_text = """
1위보다 재미있는 2위 싸움…한화생명이냐 T1이냐
입력2025.08.19. 오후 2:41
수정2025.08.19. 오후 2:42
기사원문

공감
7
텍스트 음성 변환 서비스
글자 크기 변경
공유하기
LCK 5라운드 1주 차 대진 눈길
'승승장구' 젠지 만나는 한화생명
연승 달리는 T1, 2위 탈환할까

2025 LCK 정규 리그 2위 도약을 노리는 T1. 라이엇 게임즈 제공

젠지가 독주하는 올해 LCK에서 한화생명e스포츠와 T1의 2위 싸움이 단 1점 차이의 흥미로운 양상으로 전개되고 있다.

라이엇 게임즈가 개발·서비스하는 '리그 오브 레전드' e스포츠 한국 프로 리그를 주최하는 LCK는 2025 LCK 5라운드 1주 차가 20일부터 24일까지 서울 종로구 그랑서울 롤파크 LCK 아레나에서 진행된다고 19일 밝혔다.

현재 젠지가 25승 1패로 최강자 면모를 과시하고 있는데, 한화생명e스포츠와 T1은 18승 8패로 승패가 동률이다. 득실 차에서 1점이 앞선 한화생명이 2위를 지키고 있어 다가오는 경기에서 한 번이라도 지면 순위가 바뀔 수 있는 상황이다.

비교적 수월한 상대를 만나는 T1과 달리 한화생명은 강적 젠지와 맞붙게 됐다.

한화생명은 21일 kt 롤스터, 23일 젠지와 대결을 펼친다. 3라운드부터 한화생명은 젠지와 T1 등 레전드 그룹 상위권 팀들에게 한 번도 승리하지 못했다. 이번 젠지와의 경기에서 또 패배하면 2위 수성의 가능성은 희박해진다.

상승세를 이어가고 있는 T1은 4라운드 젠지와의 대결을 제외하면 3연승을 달리고 있다. 레전드 그룹 1위와 2위에게는 플레이오프 2라운드 직행권이 주어지는 만큼, 시즌 내내 3위였던 T1 입장에서 반드시 빼앗아야 하는 자리다.

T1은 22일 농심 레드포스와 24일 kt 롤스터를 상대한다. T1은 3라운드부터 두 팀을 상대로 모두 2대 0 승리를 거뒀다.

다만 두 팀이 이번 주 경기에서 같은 승수를 기록한다면 세트 득실에 따라 2위와 3위 자리가 결정된다. 한화생명과 T1 모두 승리뿐만 아니라 2대 0 완승을 해야 유리하다.

2025 LCK 3~5라운드는 레전드 그룹과 라이즈 그룹으로 나뉘어 진행된다. 유튜브 LCK 채널, SOOP, 네이버 이스포츠&게임 등에서 생중계한다.

정길준 기자 kjkj@edaily.co.kr
"""

# 6. 체인 실행 및 결과 출력
# invoke() 메서드를 호출하여 전체 체인을 실행하고, 파싱된 Pydantic 객체를 반환받습니다.
parsed_output = chain.invoke(
    {
        "article_text": article_text,
        "format_instructions": parser.get_format_instructions()
    }
)

print("=== 파싱된 객체 정보 ===")
print(f"객체 타입: {type(parsed_output)}")
print(f"기사 제목: {parsed_output.article_title}")
print(f"주요 토픽: {parsed_output.main_topic}")
print("\n=== 현재 순위 ===")
for team in parsed_output.current_standings:
    print(f"팀: {team.team}, 순위: {team.rank}, 승: {team.wins}, 패: {team.losses}, 세트 득실: {team.score_difference}")

print("\n=== 다가오는 경기 ===")
for match in parsed_output.upcoming_matches:
    print(f"경기: {match.team} vs {match.opponent}, 날짜: {match.date}")



=== 파싱된 객체 정보 ===
객체 타입: <class '__main__.LCKArticleSummary'>
기사 제목: 1위보다 재미있는 2위 싸움…한화생명이냐 T1이냐
주요 토픽: LCK 5라운드 1주 차 대진

=== 현재 순위 ===
팀: 젠지, 순위: 1, 승: 25, 패: 1, 세트 득실: None
팀: 한화생명e스포츠, 순위: 2, 승: 18, 패: 8, 세트 득실: 1점
팀: T1, 순위: 2, 승: 18, 패: 8, 세트 득실: None

=== 다가오는 경기 ===
경기: 한화생명e스포츠 vs kt 롤스터, 날짜: 2025.08.21
경기: 한화생명e스포츠 vs 젠지, 날짜: 2025.08.23
경기: T1 vs 농심 레드포스, 날짜: 2025.08.22
경기: T1 vs kt 롤스터, 날짜: 2025.08.24


### StructuredOutputParser을 이용한 운동 기록 정리

In [17]:
# 1. Pydantic 모델 정의
# 원하는 출력 구조를 모델로 만듭니다.
from pydantic import BaseModel, Field

class Workout(BaseModel):
    date: str = Field(description="운동 날짜")
    type: str = Field(description="운동 종류 (예: 달리기, 걷기, 헬스)")
    duration_minutes: int = Field(description="운동 시간 (분 단위)")
    calories_burned: int = Field(description="소모 칼로리 (단위: kcal)")

# 2. OutputParser와 프롬프트 템플릿 설정
# Pydantic 모델을 파서에 전달하여 출력 형식을 정의합니다.
from langchain.output_parsers import PydanticOutputParser

class StructuredOutputParser(PydanticOutputParser):
    @classmethod
    def from_pydantic(cls, pydantic_object):
        return cls(pydantic_object=pydantic_object)

parser = StructuredOutputParser.from_pydantic(pydantic_object=Workout)
format_instructions = parser.get_format_instructions()

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""사용자의 운동 기록을 분석하여 아래 형식에 맞춰 출력해.
    
    {format_instructions}
    
    운동 기록:
    {input_text}""",
    input_variables=["input_text"],
    partial_variables={"format_instructions": format_instructions}
)

# 3. 모델 설정 및 체인 연결
# 파서와 프롬프트, 모델을 연결합니다.
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = LLMChain(prompt=prompt, llm=model, output_parser=parser)

# 4. 운동 기록 입력 및 결과 출력
workout_log = "2025년 8월 19일에 1시간 30분 동안 헬스를 했고, 250 칼로리를 소모했어."

# LCEL을 사용하여 체인을 실행합니다.
output = chain.invoke({"input_text": workout_log})

print(output)
print(type(output))

{'input_text': '2025년 8월 19일에 1시간 30분 동안 헬스를 했고, 250 칼로리를 소모했어.', 'text': Workout(date='2025-08-19', type='헬스', duration_minutes=90, calories_burned=250)}
<class 'dict'>
